In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

import setting

path_dict = path(setting)

In [ ]:
nmf_map = ccal.load_nmf_map(path_dict['nmf_map_file_path'])

nmf_map.plot_nmf_map(
    'h',
    element_marker_size=setting.NMF_MAP_H_ELEMENT_MARKER_SIZE,
)

In [ ]:
w = pd.DataFrame(
    nmf_map.wt.T,
    index=nmf_map.wt_elements,
    columns=nmf_map.nodes,
)

In [ ]:
feature_x_sample = pd.read_table(
    path_dict['feature_x_sample_file_path'],
    index_col=0,
)

feature_context_matrix = pd.read_table(
    path_dict['feature_context_matrix_file_path'],
    index_col=0,
)

sample_context_matrix = pd.read_table(
    path_dict['sample_context_matrix_file_path'],
    index_col=0,
).T

selected_signal_matrix = pd.read_table(
    path_dict['selected_signal_matrix_file_path'],
    index_col=0,
)

In [ ]:
for name, feature_x_sample_ in (
    (
        'Feature Context Matrix',
        feature_context_matrix,
    ),
    (
        'Sample Context Matrix',
        sample_context_matrix,
    ),
    (
        'Selected Signal Matrix',
        selected_signal_matrix,
    ),
):

    if name == 'Selected Signal Matrix':

        matrix = feature_x_sample_ 

    else:

        negative_feature_x_sample = -feature_x_sample_.clip(upper=0)

        negative_feature_x_sample.index = feature_x_sample_.index.map(lambda str_: '(-) {}'.format(str_))

        positive_feature_x_sample = feature_x_sample_.clip(lower=0)

        positive_feature_x_sample.index = feature_x_sample_.index.map(lambda str_: '(+) {}'.format(str_))

        w_negative_features = w.index[w.index.str.startswith('(-)')]

        common_negative_features = w_negative_features & negative_feature_x_sample.index

        if w_negative_features.size:
            
            print('(-) common features: {}/{} ({:.2f}%)'.format(
                common_negative_features.size,
                w_negative_features.size,
                100 * common_negative_features.size / w_negative_features.size,
            ))

        w_positive_features = w.index[w.index.str.startswith('(+)')]

        common_positive_features = w_positive_features & positive_feature_x_sample.index

        if w_positive_features.size:
            
            print('(+) common features: {}/{} ({:.2f}%)'.format(
                common_positive_features.size,
                w_positive_features.size,
                100 * common_positive_features.size / w_positive_features.size,
            ))

        matrix = pd.concat((
            negative_feature_x_sample.loc[common_negative_features],
            positive_feature_x_sample.loc[common_positive_features],
        ))

    h_solved = ccal.solve_for_nmf_h(
        matrix,
        w.loc[matrix.index],
        method='nnls',
    )
    
    title = 'Project {}'.format(name)

    predicted_element_states = nmf_map.predict(
        'h',
        h_solved,
        title=title,
        element_marker_size=setting.NMF_MAP_H_ELEMENT_MARKER_SIZE,
    )
    
    if predicted_element_states is not None:

        ccal.plot_heat_map(
            h_solved,
            normalization_axis=0,
            normalization_method='-0-',
            column_annotation=predicted_element_states,
            cluster_axis=1,
            column_annotation_colors=tuple(nmf_map.h_state_colors[i] for i in sorted(predicted_element_states.unique())),
            title=title,
            xaxis_title=h_solved.columns.name,
            yaxis_title=h_solved.index.name,
        )